# Projet de Machine Learning

In [124]:
from tools import *
import math as math
import copy
%matplotlib inline

In [125]:
def gen_arti(centerx=1,centery=1,sigma=0.1,nbex=100,data_type=0,eps=0.02):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param eps: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
         #melange de 2 gaussiennes
         xpos=np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//2)
         xneg=np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//2)
         data=np.vstack((xpos,xneg))
         y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,eps,nbex)
    data[:,1]+=np.random.normal(0,eps,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

    

In [128]:
def f_K(x, e, sigma=1):
    return math.exp(-pow(np.linalg.norm(x-e), 2)/(2*pow(sigma,2)))

# Calcul de tous les H(xl) pour le a minimal 
def HA(a, S, dataset, y):
    N = dataset.shape[0]
    Kql= np.zeros((N,N))
    Kal = np.zeros((N,N))
    f = np.zeros((0, 0))
    
    #print "Calcul des fl..."
    for l,x in enumerate(dataset):
        fl = 0
        for j,e in enumerate(S):
            fl += a[j]*f_K(x,e)
        f = np.append(f, fl)
        
    #print "Done"
    for i,x in enumerate(S):
        for j,e in enumerate(S):
            Kql[i,j] = f_K(x,e)
            
    Kql = np.asmatrix(np.asarray(Kql))
    f = np.asmatrix(f.reshape((f.shape[0], 1)))
    y = np.asmatrix(y)

    
    #print "Calcul de H(xl)..."
    return -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a

# Calcul le a minimal approximé par Newton-R
def H(x, S, a_prec, dataset, y):

    a = a_optim(a_prec) 
    return HA(a, S, dataset, y), a

def a_optim(a_prec):

    f = K*a_prec
    print "f", max(f) 
    p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
    d = np.asarray(p*(1-p))
    W = np.asmatrix(np.zeros((d.shape[0], d.shape[0])))
    
    for i,element in enumerate(d):
        W[i,i] = element
        
    z = K*a_prec+np.linalg.inv(W)*(y-p)
    return np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 

def ivm(dataset, y):

    N = dataset.shape[0]
    a = np.zeros((N,1))
    R = np.zeros((N,dataset.shape[1]))
    S = []
    lamda = 3
    a_s = None
    
    Hk = 1
    Hk_1 = 0
    for i, e in enumerate(dataset):
        R[i] = e

    while abs(Hk-Hk_1)/abs(Hk) > 0.01:
        if R.shape[0] == 0:
            break
            
        #print "Calcul des nouveaux H(xl)..."
        h = np.ones((R.shape[0], 1))
        S_b = []
        for e in S:
            S_b.append(e)
            
        A = []
        for i, xl in enumerate(R):
            S_b.append(xl)
            h[i], a = H(xl, S_b, a, dataset, y)
            A.append(a)
            S_b.pop()
            
            
        #print "Done"
        #print "On recupere le plus petit H(xl)"
        xl_opt = np.argmin(h)
        Hk_1 = Hk
        Hk = h[xl_opt]
        #print h
        #print "Le plus petit est ", Hk, " il est à l'indice : ", xl_opt

        #print "On l'ajoute a S..."

        S.append(R[xl_opt])
        #print "--------------> S:"

        #print "OK"
        #print "On l'enleve a R..."

        data = []
        for i, e in enumerate(R):
            if i == xl_opt:
                pass
            else:
                data.append(e)
        R = np.asarray(data)
        # update de a
        print "Updating a..."

        a = a_optim(A[xl_opt])
        #print "On garde celui associe au meilleur:", A[xl_opt]
        #print "OK"
        print "ecart", abs(Hk-Hk_1)/abs(Hk)
        
    print "S final"
    for el in S:
        print el
    print "Il y a ", len(S), " import points"


In [129]:
data, y = gen_arti()
data = np.asarray(data)
print data
N = data.shape[0]
K = np.asmatrix(np.zeros((N,N)))
lamda=1
for i,x in enumerate(data):
    for j,e in enumerate(data):
        K[i,j] = f_K(x,e)

y = y.reshape((y.shape[0], 1))
ivm(data, y)

[[ 0.80083979  0.76225576]
 [-1.00846237 -1.27688136]
 [-1.2118586  -0.60517197]
 [ 1.47576448  0.96165324]
 [-0.55145548 -0.94102777]
 [ 1.19597569  1.16013876]
 [ 1.08427465  1.20208901]
 [ 0.79416161  1.14958213]
 [ 0.47137696  1.12582011]
 [-0.97367509 -0.82477624]
 [-1.62476199 -0.86762858]
 [ 0.43661165  0.92773669]
 [-0.74596145 -1.08502089]
 [ 0.81467817  0.43854168]
 [ 1.02960114  0.52739333]
 [ 0.9472117   1.21876636]
 [-1.33866025 -0.74011301]
 [-0.82756204 -1.28138345]
 [-1.05787879 -1.39827256]
 [-0.86650136 -1.22478179]
 [-1.51494472 -1.03682879]
 [ 1.19012758  0.99872033]
 [ 0.84278812  0.6100098 ]
 [ 1.54952251  1.34388438]
 [-0.25123719 -0.84385685]
 [-1.24913572 -0.82305272]
 [ 1.4522101   1.58120363]
 [-1.01404022 -1.07310065]
 [-1.40564379 -1.54118031]
 [ 1.574901    1.22095993]
 [-1.091651   -1.22852135]
 [-1.02583184 -1.14871213]
 [ 1.30930673  1.21198819]
 [ 0.83793384  1.08777377]
 [-1.55105069 -1.04783784]
 [ 1.16490147  0.77679228]
 [ 0.48131873  0.89392382]
 